# Create Custom Sentinel-2 (COG) Tiler

Note: See https://github.com/developmentseed/titiler-pds for a end-to-end implementation

### Sentinel 2

Thanks to Digital Earth Africa and in collaboration with Sinergise, Element 84, Amazon Web Services (AWS) and the Committee on Earth Observation Satellites (CEOS), Sentinel 2 (Level 2) data over Africa, usually stored as JPEG2000, has been translated to COG more important a STAC database and API has been setup. 

https://www.digitalearthafrica.org/news/operational-and-ready-use-satellite-data-now-available-across-africa

The API is provided by [@element84](https://www.element84.com) and follows the latest specification: https://earth-search.aws.element84.com/v0


## Requirements

To be able to run this notebook you'll need the following requirements:
- rasterio
- ipyleaflet
- requests
- titiler
- uvicorn
- rio-tiler-pds
- cogeo-mosaic

`!pip install titiler["server"] cogeo-mosaic ipyleaflet requests rio-tiler-pds`

## 1. Create the CustomTiler

In order to create our custom tiler we will use the rio-tiler-pds `S2COGReader` and the different Tiler Factories from titiler.

```python
"""
Sentinel 2 (COG) Tiler.

Based on rio-tiler-pds and titiler

>>> pip install titiler rio-tiler-pds

"""
from dataclasses import dataclass, field
from typing import Dict, Type, Optional, Sequence

from titiler.endpoints.factory import TilerFactory, MosaicTilerFactory
from titiler.dependencies import DefaultDependency
from titiler.models.dataset import Info, Metadata

from rio_tiler_pds.sentinel.aws import S2COGReader
from rio_tiler_pds.sentinel.utils import s2_sceneid_parser

from fastapi import FastAPI, Depends, Query


@dataclass
class CustomPathParams:
    """Create dataset path from args"""

    sceneid: str = Query(..., description="Landsat 8 Sceneid.")
    scene_metadata: Dict = field(init=False)

    def __post_init__(self,):
        """Define dataset URL."""
        self.url = self.sceneid
        self.scene_metadata = s2_sceneid_parser(self.sceneid)


def BandsParams(
    bands: str = Query(
        ...,
        title="bands names",
        description="comma (',') delimited bands names.",
    )
) -> Sequence[str]:
    """Bands."""
    return bands.split(",")


@dataclass
class BandsExprParams(DefaultDependency):
    """Band names and Expression parameters."""

    bands: Optional[str] = Query(
        None,
        title="bands names",
        description="comma (',') delimited bands names.",
    )
    expression: Optional[str] = Query(
        None,
        title="Band Math expression",
        description="rio-tiler's band math expression.",
    )

    def __post_init__(self):
        """Post Init."""
        if self.bands is not None:
            self.kwargs["bands"] = self.bands.split(",")
        if self.expression is not None:
            self.kwargs["expression"] = self.expression


@dataclass
class S2COGTiler(TilerFactory):
    """Custom Tiler Class for STAC."""

    reader: Type[S2COGReader] = field(default=S2COGReader)

    path_dependency: Type[CustomPathParams] = CustomPathParams

    layer_dependency: Type[DefaultDependency] = BandsExprParams

    def info(self):
        """Register /info endpoint."""

        @self.router.get(
            "/info",
            response_model=Info,
            response_model_exclude={"minzoom", "maxzoom", "center"},
            response_model_exclude_none=True,
            responses={200: {"description": "Return dataset's basic info."}},
        )
        def info(
            src_path=Depends(self.path_dependency),
            bands=Depends(BandsParams),
            kwargs: Dict = Depends(self.additional_dependency),
        ):
            """Return basic info."""
            with self.reader(src_path.url, **self.reader_options) as src_dst:
                info = src_dst.info(bands=bands, **kwargs)
            return info

    def metadata(self):
        """Register /metadata endpoint."""

        @self.router.get(
            "/metadata",
            response_model=Metadata,
            response_model_exclude={"minzoom", "maxzoom", "center"},
            response_model_exclude_none=True,
            responses={200: {"description": "Return dataset's metadata."}},
        )
        def metadata(
            src_path=Depends(self.path_dependency),
            bands=Depends(BandsParams),
            metadata_params=Depends(self.metadata_dependency),
            kwargs: Dict = Depends(self.additional_dependency),
        ):
            """Return metadata."""
            with self.reader(src_path.url, **self.reader_options) as src_dst:
                info = src_dst.metadata(
                    metadata_params.pmin,
                    metadata_params.pmax,
                    bands=bands,
                    **metadata_params.kwargs,
                    **kwargs,
                )
            return info


app = FastAPI()


scene_tiler = S2COGTiler(router_prefix="scenes")
app.include_router(scene_tiler.router, prefix="/scenes", tags=["scenes"])

mosaic_tiler = MosaicTilerFactory(
    router_prefix="mosaic",
    dataset_reader=S2COGReader,
    layer_dependency=BandsExprParams,
    add_update=False,
    add_create=False,
)
app.include_router(mosaic_tiler.router, prefix="/mosaic", tags=["mosaic"])
```

## 2. Launch the tiler

Save the python code form 1. into a file named app.py and then start the server within another terminal

```
$ uvicorn app:app
```

## 3. Search Data

In [1]:
import os
import json
import base64
import requests
import datetime
import itertools
import urllib.parse
import pathlib

from io import BytesIO
from functools import partial
from concurrent import futures

from rasterio.plot import reshape_as_image
from rasterio.features import bounds as featureBounds

from ipyleaflet import Map, basemaps, TileLayer, basemap_to_tiles, GeoJSON

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Endpoint variables
titiler_endpoint = "http://127.0.0.1:8000"
stac_endpoint = "https://earth-search.aws.element84.com/v0/search"

# Make sure both are up
assert requests.get(f"{titiler_endpoint}/docs").status_code == 200
assert requests.get(stac_endpoint).status_code == 200

More info: https://github.com/radiantearth/stac-api-spec for more documentation about the stac API

1. AOI

You can use geojson.io to define your search AOI

In [3]:
geojson = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -2.83447265625,
              4.12728532324537
            ],
            [
              2.120361328125,
              4.12728532324537
            ],
            [
              2.120361328125,
              8.254982704877875
            ],
            [
              -2.83447265625,
              8.254982704877875
            ],
            [
              -2.83447265625,
              4.12728532324537
            ]
          ]
        ]
      }
    }
  ]
}

bounds = featureBounds(geojson)

m = Map(
    basemap=basemaps.OpenStreetMap.Mapnik,
    center=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom=6
)

geo_json = GeoJSON(data=geojson)
m.add_layer(geo_json)
m

Map(center=[6.191134014061623, -0.3570556640625], controls=(ZoomControl(options=['position', 'zoom_in_text', '…

2. Define dates and other filters

In [91]:
start = datetime.datetime.strptime("2019-01-01", "%Y-%m-%d").strftime("%Y-%m-%dT00:00:00Z")
end = datetime.datetime.strptime("2019-12-11", "%Y-%m-%d").strftime("%Y-%m-%dT23:59:59Z")

# POST body
query = {
    "collections": ["sentinel-s2-l2a-cogs"],
    "datetime": f"{start}/{end}",
    "query": {
        "eo:cloud_cover": {
            "lt": 3
        },
        "sentinel:data_coverage": {
            "gt": 10
        }
    },
    "intersects": geojson["features"][0]["geometry"],
    "limit": 1000,
    "fields": {
      'include': ['id', 'properties.datetime', 'properties.eo:cloud_cover'],  # This will limit the size of returned body
      'exclude': ['assets', 'links']  # This will limit the size of returned body
    },
    "sortby": [
        {
            "field": "properties.eo:cloud_cover",
            "direction": "desc"
        },
    ]
}

# POST Headers
headers = {
    "Content-Type": "application/json",
    "Accept-Encoding": "gzip",
    "Accept": "application/geo+json",
}

data = requests.post(stac_endpoint, headers=headers, json=query).json()
print("Results context:")
print(data["context"])

sceneid = [f["id"] for f in data["features"]]
cloudcover = [f["properties"]["eo:cloud_cover"] for f in data["features"]]
dates = [f["properties"]["datetime"][0:10] for f in data["features"]]

Results context:
{'page': 1, 'limit': 1000, 'matched': 380, 'returned': 380}


In [92]:
data["features"][0]

{'bbox': [0.2927641695171779,
  4.429965030479934,
  1.2856914562725514,
  5.4257779507944806],
 'geometry': {'coordinates': [[[0.2967780962947474, 4.429965030479934],
    [0.2927641695171779, 5.422153918603194],
    [1.28314366979741, 5.4257779507944806],
    [1.2856914562725514, 4.43292307525344],
    [0.2967780962947474, 4.429965030479934]]],
  'type': 'Polygon'},
 'id': 'S2B_31NBF_20190109_0_L2A',
 'collection': 'sentinel-s2-l2a-cogs',
 'type': 'Feature',
 'properties': {'datetime': '2019-01-09T10:29:38Z', 'eo:cloud_cover': 2.98}}

In [93]:
m = Map(
    basemap=basemaps.OpenStreetMap.Mapnik,
    center=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom=8
)

geo_json = GeoJSON(
    data=data,
    style={
        'opacity': 1, 'dashArray': '1', 'fillOpacity': 0, 'weight': 1
    },
)
m.add_layer(geo_json)
m

Map(center=[6.305019725812759, 0.044313056455806965], controls=(ZoomControl(options=['position', 'zoom_in_text…

## 4. Visualize One Item

In [94]:
# Print what band are available
from rio_tiler_pds.sentinel.aws.sentinel2 import default_l2a_bands
print(default_l2a_bands)

('B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B11', 'B12', 'B8A')


In [95]:
# Fetch TileJSON
# For this example we use the first `sceneid` return from the STAC API
# and we sent the Bands to B04,B03,B02 which are red,green,blue
data = requests.get(f"{titiler_endpoint}/scenes/tilejson.json?sceneid={sceneid[4]}&bands=B04,B03,B02&rescale=0,2000").json()
print(data)

{'tilejson': '2.2.0', 'name': 'S2B_31NCG_20190926_0_L2A', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['http://127.0.0.1:8000/scenes/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?sceneid=S2B_31NCG_20190926_0_L2A&bands=B04%2CB03%2CB02&rescale=0%2C2000'], 'minzoom': 8, 'maxzoom': 14, 'bounds': [1.1920033127639242, 5.337108286531346, 2.1859046942193197, 6.332392577649882], 'center': [1.688954003491622, 5.834750432090614, 8]}


In [96]:
bounds = data["bounds"]
m = Map(
    center=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom=10
)

tiles = TileLayer(
    url=data["tiles"][0],
    min_zoom=data["minzoom"],
    max_zoom=data["maxzoom"],
    opacity=1
)
m.add_layer(tiles)
m

Map(center=[5.834750432090614, 1.688954003491622], controls=(ZoomControl(options=['position', 'zoom_in_text', …

## 5. Mosaic

Working with 1 scene is great, but working with 10s is better. Lets see how to create mosaics and use it with our new tiler.

### 5.1 Create a MosaicJSON document


To create a mosaicJSON, we can use developmentseed/cogeo-mosaic python module. Within this module there is a `backend` which is able to work with STAC API directly to create mosaicJSON.

In [97]:
from cogeo_mosaic.backends import MosaicBackend


def custom_accessor(feature):
    """Return feature identifier."""
    return feature["id"]

with MosaicBackend(
    "stac+https://earth-search.aws.element84.com/v0/search",
    query,
    minzoom=8,
    maxzoom=15,
    backend_options={"accessor": custom_accessor},
) as mosaic:
    print(mosaic.metadata)
    mosaic_doc = mosaic.mosaic_def.dict(exclude_none=True)

{'mosaicjson': '0.0.2', 'version': '1.0.0', 'minzoom': 8, 'maxzoom': 15, 'quadkey_zoom': 8, 'bounds': [-3.000172884686006, 3.5633051316122546, 3.0887989975976198, 9.046734320013263], 'center': (0.044313056455806965, 6.305019725812759, 8)}


In [98]:
mosaic_doc

{'mosaicjson': '0.0.2',
 'version': '1.0.0',
 'minzoom': 8,
 'maxzoom': 15,
 'quadkey_zoom': 8,
 'bounds': [-3.000172884686006,
  3.5633051316122546,
  3.0887989975976198,
  9.046734320013263],
 'center': (0.044313056455806965, 6.305019725812759, 8),
 'tiles': {'03333103': ['S2A_30PWQ_20191126_0_L2A',
   'S2B_30PWQ_20190224_0_L2A',
   'S2B_30PWQ_20191201_0_L2A',
   'S2A_30PWQ_20190110_0_L2A',
   'S2A_30PWQ_20190130_0_L2A',
   'S2B_30PWQ_20190204_0_L2A',
   'S2B_30PWQ_20190115_0_L2A',
   'S2B_30PWQ_20190214_0_L2A',
   'S2B_30PWQ_20190326_0_L2A',
   'S2A_30PWQ_20190410_0_L2A'],
  '03333112': ['S2A_30PXQ_20191203_0_L2A',
   'S2A_30PXQ_20191113_0_L2A',
   'S2A_30PWQ_20190226_0_L2A',
   'S2A_30PWQ_20191123_0_L2A',
   'S2A_30PXQ_20190507_0_L2A',
   'S2A_30PWQ_20191126_0_L2A',
   'S2A_30PXQ_20190107_0_L2A',
   'S2A_30PWQ_20191113_0_L2A',
   'S2B_30PWQ_20190224_0_L2A',
   'S2B_30PWQ_20191201_0_L2A',
   'S2B_30PXQ_20191128_0_L2A',
   'S2B_30PWQ_20190502_0_L2A',
   'S2A_30PWQ_20190107_0_L2A',
  

### 5.2 Custom Filter



As we can see in the mosaicJSON, each `quadkey` index has 10s scenes. Because we want to build a visual mosaic, we don't need as many assets and maybe 1 or 2 scene per quadkey will be enought to create mercator tiles. To reduce the number of scenes per quadkey we need to apply a custom filter while building the mosaic.

In [99]:
from typing import Dict, List, Sequence, Optional
from pygeos import polygons
import mercantile


def optimized_filter(
    tile: mercantile.Tile,  # noqa
    dataset: Sequence[Dict],
    geoms: Sequence[polygons],
    minimum_tile_cover=None,  # noqa
    tile_cover_sort=False,  # noqa
    maximum_items_per_tile: Optional[int] = None,
) -> List:    
    """Optimized filter that keeps only one item per grid ID."""
    gridid: List[str] = []
    selected_dataset: List[Dict] = []
    
    for item in dataset:
        grid = item["id"].split("_")[1]
        if grid not in gridid:
            gridid.append(grid)
            selected_dataset.append(item)

    dataset = selected_dataset

    indices = list(range(len(dataset)))
    if maximum_items_per_tile:
        indices = indices[:maximum_items_per_tile]

    return [dataset[ind] for ind in indices]



with MosaicBackend(
    "stac+https://earth-search.aws.element84.com/v0/search",
    optquery,
    minzoom=8,
    maxzoom=14,
    backend_options={"accessor": custom_accessor, "asset_filter": optimized_filter},
) as mosaic:
    print(mosaic.metadata)
    mosaic_doc = mosaic.mosaic_def.dict(exclude_none=True)

{'mosaicjson': '0.0.2', 'version': '1.0.0', 'minzoom': 8, 'maxzoom': 14, 'quadkey_zoom': 8, 'bounds': [-3.000172884686006, 3.5633051316122546, 3.0887989975976198, 9.046734320013263], 'center': (0.044313056455806965, 6.305019725812759, 8)}


In [100]:
mosaic_doc

{'mosaicjson': '0.0.2',
 'version': '1.0.0',
 'minzoom': 8,
 'maxzoom': 14,
 'quadkey_zoom': 8,
 'bounds': [-3.000172884686006,
  3.5633051316122546,
  3.0887989975976198,
  9.046734320013263],
 'center': (0.044313056455806965, 6.305019725812759, 8),
 'tiles': {'03333103': ['S2B_30PWQ_20191201_0_L2A'],
  '03333112': ['S2A_30PXQ_20191203_0_L2A', 'S2B_30PWQ_20191201_0_L2A'],
  '03333113': ['S2A_30PXQ_20191203_0_L2A',
   'S2B_30PYQ_20191128_0_L2A',
   'S2A_30PZQ_20191123_0_L2A'],
  '12222002': ['S2A_31PBK_20191210_0_L2A',
   'S2A_31PCK_20191210_0_L2A',
   'S2A_30PZQ_20191123_0_L2A'],
  '12222003': ['S2A_31PCK_20191210_0_L2A', 'S2A_31PDK_20191210_0_L2A'],
  '12222012': ['S2A_31PDK_20191210_0_L2A'],
  '03333121': ['S2B_30PWQ_20191201_0_L2A',
   'S2B_30NWN_20190326_0_L2A',
   'S2B_30NWP_20190326_0_L2A'],
  '03333130': ['S2B_30NWN_20191208_0_L2A',
   'S2B_30NXN_20191208_0_L2A',
   'S2B_30NXP_20191208_0_L2A',
   'S2A_30PXQ_20191203_0_L2A',
   'S2B_30PWQ_20191201_0_L2A',
   'S2B_30NWP_20190326_

### 5.3 Save Mosaic

In [101]:
mosaic_file = "mymosaic.json.gz"
with MosaicBackend(mosaic_file, mosaic_def=mosaic_doc) as mosaic:
    mosaic.write(overwrite=True)

In [102]:
# We need to pass absolute path to the tiler
mosaic = str(pathlib.Path(mosaic_file).absolute())

### 5.4 Use It

In [103]:
data = requests.get(f"{titiler_endpoint}/mosaic/info?url=file:///{mosaic}").json()
print(data)

{'bounds': [-3.000172884686006, 3.5633051316122546, 3.0887989975976198, 9.046734320013263], 'center': [0.044313056455806965, 6.305019725812759, 8], 'minzoom': 8, 'maxzoom': 14, 'name': 'mosaic', 'quadkeys': ['03333103', '03333112', '03333113', '12222002', '12222003', '12222012', '03333121', '03333130', '03333131', '12222020', '12222021', '12222030', '03333123', '03333132', '03333133', '12222022', '12222023', '12222032', '03333301', '03333310', '03333311', '12222200', '12222201', '12222210', '03333312', '03333313', '12222202', '12222203']}


In [104]:
data = requests.get(f"{titiler_endpoint}/mosaic/tilejson.json?url=file:///{mosaic}&bands=B01&rescale=0,1000").json()

bounds = data["bounds"]
m = Map(
    center=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom=10
)

tiles = TileLayer(
    url=data["tiles"][0],
    min_zoom=data["minzoom"],
    max_zoom=data["maxzoom"],
    opacity=1
)
m.add_layer(tiles)
m

Map(center=[6.305019725812759, 0.044313056455806965], controls=(ZoomControl(options=['position', 'zoom_in_text…